In [13]:
import numpy as np
from scipy.optimize import minimize

AU_KM = 1.495978707e8

def R3(theta):
    c, s = np.cos(theta), np.sin(theta)
    return np.array([[c, -s, 0.0],
                     [s,  c, 0.0],
                     [0.0, 0.0, 1.0]])

def R1(theta):
    c, s = np.cos(theta), np.sin(theta)
    return np.array([[1.0, 0.0, 0.0],
                     [0.0,  c, -s],
                     [0.0,  s,  c]])

def Q_matrix(i, w, Om):
    return R3(Om) @ R1(i) @ R3(w)

def make_elems(a_km, e, i_deg, w_deg, Om_deg):
    i = np.deg2rad(i_deg)
    w = np.deg2rad(w_deg)
    Om = np.deg2rad(Om_deg)
    Q = Q_matrix(i, w, Om)
    return (a_km, e, i, w, Om, Q)

def r_ellipse(elems, f):
    a, e, i, w, Om, Q = elems
    r = a * (1.0 - e**2) / (1.0 + e * np.cos(f))
    rp = np.array([r * np.cos(f), r * np.sin(f), 0.0])
    return Q @ rp

def r_hyperbola_from_F(elems, F):
    a, e, i, w, Om, Q = elems
    a_mag = abs(a)
    x = a_mag * (e - np.cosh(F))
    y = a_mag * np.sqrt(e**2 - 1.0) * np.sinh(F)

    rp = np.array([x, y, 0.0])
    return Q @ rp

def moid_ellipse_ellipse(e1, e2, n=360, refine_k=20):
    fgrid = np.linspace(0.0, 2.0*np.pi, n, endpoint=False)

    R1s = np.stack([r_ellipse(e1, f) for f in fgrid])
    R2s = np.stack([r_ellipse(e2, f) for f in fgrid])

    diff = R1s[:, None, :] - R2s[None, :, :]
    d2 = np.einsum("ijk,ijk->ij", diff, diff)

    flat_idx = np.argpartition(d2.ravel(), refine_k)[:refine_k]
    candidates = []
    for idx in flat_idx:
        i = idx // n
        j = idx % n
        candidates.append((fgrid[i], fgrid[j]))

    bounds = [(0.0, 2.0*np.pi), (0.0, 2.0*np.pi)]

    def obj(x):
        f1, f2 = x
        d = r_ellipse(e1, f1) - r_ellipse(e2, f2)
        return float(d @ d)

    best_fun = np.inf
    best_x = None

    for f1_0, f2_0 in candidates:
        res = minimize(obj, x0=np.array([f1_0, f2_0]), method="L-BFGS-B", bounds=bounds)
        if res.fun < best_fun:
            best_fun = res.fun
            best_x = res.x

    dmin = np.sqrt(best_fun)
    return dmin, best_x[0], best_x[1]

def moid_ellipse_hyperbola(e_ell, e_hyp, n_f=360, n_F=360, Fmax=2.0, refine_k=24):
    fgrid = np.linspace(0.0, 2.0*np.pi, n_f, endpoint=False)
    Fgrid = np.linspace(-Fmax, Fmax, n_F)

    R1s = np.stack([r_ellipse(e_ell, f) for f in fgrid])
    R2s = np.stack([r_hyperbola_from_F(e_hyp, F) for F in Fgrid])

    diff = R1s[:, None, :] - R2s[None, :, :]
    d2 = np.einsum("ijk,ijk->ij", diff, diff)

    flat_idx = np.argpartition(d2.ravel(), refine_k)[:refine_k]
    candidates = []
    for idx in flat_idx:
        i = idx // n_F
        j = idx % n_F
        candidates.append((fgrid[i], Fgrid[j]))

    bounds = [(0.0, 2.0*np.pi), (-Fmax, Fmax)]

    def obj(x):
        f, F = x
        d = r_ellipse(e_ell, f) - r_hyperbola_from_F(e_hyp, F)
        return float(d @ d)

    best_fun = np.inf
    best_x = None

    for f0, F0 in candidates:
        res = minimize(obj, x0=np.array([f0, F0]),
                       method="L-BFGS-B", bounds=bounds)
        if res.fun < best_fun:
            best_fun = res.fun
            best_x = res.x

    dmin = np.sqrt(best_fun)
    return dmin, best_x[0], best_x[1]

earth   = make_elems( 1.4765067e8, 9.1669995e-3, 4.2422693e-3, 6.64375167e1, 1.4760836e1)
apophis = make_elems( 1.3793939e8, 1.9097084e-1, 3.3356539e0,  1.2919949e2,  2.0381969e2)
yr4     = make_elems( 3.7680703e8, 6.6164147e-1, 3.4001497e0,  1.3429905e2,  2.7147904e2)
atlas   = make_elems(-3.9552667e7, 6.1469268e0,  1.7512507e2,  1.2817255e2,  3.2228906e2)

d1, fE1, fA1 = moid_ellipse_ellipse(earth, apophis)
d2, fE2, fY2 = moid_ellipse_ellipse(earth, yr4)
d3, fA3, fY3 = moid_ellipse_ellipse(apophis, yr4)
d4, fE4, F4  = moid_ellipse_hyperbola(earth, atlas, Fmax=2.0)

print("Earth - Apophis:", d1, d1 / AU_KM, fE1, fA1)
print("Earth - 2024 YR4:", d2, d2 / AU_KM, fE2, fY2)
print("Apophis - 2024 YR4:", d3, d3 / AU_KM, fA3, fY3)
print("Earth - 3I/ATLAS:", d4, d4 / AU_KM, fE4, F4)

Earth - Apophis: 847896.6685685192 0.00566783915172744 2.2334527943345512 4.121681147392492
Earth - 2024 YR4: 264452.6442742242 0.0017677567403653173 0.2081653087836902 0.8264167705989232
Apophis - 2024 YR4: 7530657.644112704 0.05033933711004822 2.1693927896535325 0.8989494516178956
Earth - 3I/ATLAS: 56610142.751206696 0.3784154312244947 1.9675023639982299 0.00323959431132693
